In [1]:
import cloudscraper
import pandas as pd
from collections import defaultdict

In [2]:
def scrape_team_stats(url, attrs):
    scraper = cloudscraper.create_scraper()
    html = scraper.get(url).text
    tables = pd.read_html(html, attrs=attrs, header=1)
    df = tables[0]
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

    return df

In [3]:
seasons = [
    "2020-2021",
    "2021-2022",
    "2022-2023",
    "2023-2024",
    "2024-2025",
    "2025-2026"
]

alldata = {}

In [4]:
def merge_squad_player_stats_per_season(data):
    merged_seasons = {}
    for season, df in data.items():
        merged = None
        for name, df in df.items():
            df = df.rename(columns={col: f"{col}" for col in df.columns if col != "Squad"})
        
            if merged is None:
                merged = df
            else:
                merged = merged.merge(df, on="Squad", how="outer")
            merged_seasons[season] = merged

    return merged_seasons

In [5]:
def clean_and_save_possession_stats(df):
    if all(col in df.columns for col in ["PrgR", "90s"]):
        df["PrgR_per90"] = (df["PrgR"] / df["90s"]).round(2)
    keep_cols = ["Squad", "Poss", "PrgR", "PrgR_per90"]
    df = df[[c for c in keep_cols if c in df.columns]]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()

    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)

    rename_cols = {
        "Poss": "Possession",
        "PrgR_per90": "Progressive_Passes_Received_Per90"
    }
    df = df.rename(columns=rename_cols)

    df = df.drop(columns=["PrgR", "90s"], errors="ignore")

    if is_against:
        df.rename(
            columns={col: col + "_Against" for col in df.columns if col != "Squad"},
            inplace=True
        )

    return df

In [6]:
# POSSESSION FOR
base_url = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs = {"id": "stats_squads_possession_for"}
for season in seasons:
    url = base_url.format(season=season)
    df = scrape_team_stats(url, attrs)
    data = clean_and_save_possession_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["possession_for"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2021-2022"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: Future

,Squad,Possession,Progressive_Passes_Received_Per90
0,Arsenal,52.8,43.21
1,Aston Villa,46.5,34.03
2,Brentford,44.8,30.79
3,Brighton,54.4,40.24
4,Burnley,40.2,25.21


In [7]:
# POSSESSION AGAINST
base_url_against = "https://fbref.com/en/comps/9/{season}/possession/Premier-League-Stats"
attrs_against = {"id": "stats_squads_possession_against"}
for season in seasons:
    url = base_url_against.format(season=season)
    df = scrape_team_stats(url, attrs_against)
    data = clean_and_save_possession_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["possession_against"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2023-2024"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/2381387787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables =

,Squad,Possession,Progressive_Passes_Received_Per90,Possession_Against,Progressive_Passes_Received_Per90_Against
0,Arsenal,58.2,54.95,41.8,24.26
1,Aston Villa,53.1,37.84,46.9,32.42
2,Bournemouth,44.4,34.11,55.6,39.08
3,Brentford,45.2,33.58,54.8,41.26
4,Brighton,59.8,46.50,40.2,30.66


In [8]:
def clean_and_save_shooting_stats(df):
    keep_cols = ["Squad", "Sh/90", "SoT/90"]
    df = df[keep_cols]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()
    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
        cols = [c for c in df.columns if c != "Squad"]
        df = df.rename(columns={c: c + "_Against" for c in cols})
    rename_cols = {
        "Sh/90": "Shots_Per_90",
        "SoT/90": "Shots_On_Target_Per_90"
    }

    df = df.rename(columns=rename_cols)
    return df

In [9]:
# SHOOTING FOR
base_url_shoot_for = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_for = {"id": "stats_squads_shooting_for"}

for season in seasons:
    url = base_url_shoot_for.format(season=season)
    df = scrape_team_stats(url, attrs_shoot_for)
    data = clean_and_save_shooting_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["shooting_for"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2023-2024"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: Future

,Squad,Possession,Progressive_Passes_Received_Per90,Possession_Against,Progressive_Passes_Received_Per90_Against,Shots_Per_90,Shots_On_Target_Per_90
0,Arsenal,58.2,54.95,41.8,24.26,17.03,5.50
1,Aston Villa,53.1,37.84,46.9,32.42,13.45,4.79
2,Bournemouth,44.4,34.11,55.6,39.08,14.18,4.61
3,Brentford,45.2,33.58,54.8,41.26,12.37,4.18
4,Brighton,59.8,46.50,40.2,30.66,14.61,5.21


In [10]:
# SHOOTING  AGAINST
base_url_shoot_against = "https://fbref.com/en/comps/9/{season}/shooting/Premier-League-Stats"
attrs_shoot_against = {"id": "stats_squads_shooting_against"}

for season in seasons:
    url = base_url_shoot_against.format(season=season)
    df = scrape_team_stats(url, attrs_shoot_against)
    data = clean_and_save_shooting_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["shooting_against"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2023-2024"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/850743466.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = p

,Squad,Possession,Progressive_Passes_Received_Per90,Possession_Against,Progressive_Passes_Received_Per90_Against,Shots_Per_90,Shots_On_Target_Per_90,Sh/90_Against,SoT/90_Against
0,Arsenal,58.2,54.95,41.8,24.26,17.03,5.50,8.42,2.11
1,Aston Villa,53.1,37.84,46.9,32.42,13.45,4.79,11.92,4.50
2,Bournemouth,44.4,34.11,55.6,39.08,14.18,4.61,13.87,4.92
3,Brentford,45.2,33.58,54.8,41.26,12.37,4.18,14.37,4.79
4,Brighton,59.8,46.50,40.2,30.66,14.61,5.21,12.16,4.18


In [11]:
def clean_and_save_attacking_stats(df):
    keep_cols = ["Squad", "SCA90", "GCA90"]
    df = df[keep_cols]

    is_against = df["Squad"].astype(str).str.contains("vs ").any()
    if is_against:
        df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
        cols = [c for c in df.columns if c != "Squad"]
        df = df.rename(columns={c: c + "_Against" for c in cols})
    rename_cols = {
        "SCA90": "Shot_Creating_Actions_Per90",
        "GCA90": "Goal_Creating_Actions_Per90"
    }

    df = df.rename(columns=rename_cols)
    return df

In [12]:
# ATTACKING FOR
base_url_att_for = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_for = {"id": "stats_squads_gca_for"}

for season in seasons:
    url = base_url_att_for.format(season=season)
    df = scrape_team_stats(url, attrs_attacking_for)
    data = clean_and_save_attacking_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["attacking_for"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2023-2024"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: Future

,Squad,Possession,Progressive_Passes_Received_Per90,Possession_Against,Progressive_Passes_Received_Per90_Against,Shots_Per_90,Shots_On_Target_Per_90,Sh/90_Against,SoT/90_Against,Shot_Creating_Actions_Per90,Goal_Creating_Actions_Per90
0,Arsenal,58.2,54.95,41.8,24.26,17.03,5.50,8.42,2.11,31.18,4.00
1,Aston Villa,53.1,37.84,46.9,32.42,13.45,4.79,11.92,4.50,24.00,3.39
2,Bournemouth,44.4,34.11,55.6,39.08,14.18,4.61,13.87,4.92,25.05,2.18
3,Brentford,45.2,33.58,54.8,41.26,12.37,4.18,14.37,4.79,21.16,2.13
4,Brighton,59.8,46.50,40.2,30.66,14.61,5.21,12.16,4.18,26.58,2.34


In [13]:
# ATTACKING AGAINST
base_url_att_against = "https://fbref.com/en/comps/9/{season}/gca/Premier-League-Stats"
attrs_attacking_against = {"id": "stats_squads_gca_against"}

for season in seasons:
    url = base_url_att_against.format(season=season)
    df = scrape_team_stats(url, attrs_attacking_against)
    data = clean_and_save_attacking_stats(df)
    if season not in alldata:
        alldata[season] = {}
    alldata[season]["attacking_against"] = data

merge = merge_squad_player_stats_per_season(alldata)
merge["2023-2024"].head()

/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, attrs=attrs, header=1)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1089070659.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Squad"] = df["Squad"].str.replace("vs ", "", regex=False)
/var/folders/dg/93wvghd5137cglm9mr9b58zh0000gn/T/ipykernel_18136/1502654688.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = 

,Squad,Possession,Progressive_Passes_Received_Per90,Possession_Against,Progressive_Passes_Received_Per90_Against,Shots_Per_90,Shots_On_Target_Per_90,Sh/90_Against,SoT/90_Against,Shot_Creating_Actions_Per90,Goal_Creating_Actions_Per90,SCA90_Against,GCA90_Against
0,Arsenal,58.2,54.95,41.8,24.26,17.03,5.50,8.42,2.11,31.18,4.00,13.97,1.16
1,Aston Villa,53.1,37.84,46.9,32.42,13.45,4.79,11.92,4.50,24.00,3.39,21.58,2.68
2,Bournemouth,44.4,34.11,55.6,39.08,14.18,4.61,13.87,4.92,25.05,2.18,24.58,2.95
3,Brentford,45.2,33.58,54.8,41.26,12.37,4.18,14.37,4.79,21.16,2.13,25.79,3.13
4,Brighton,59.8,46.50,40.2,30.66,14.61,5.21,12.16,4.18,26.58,2.34,21.42,2.50


In [14]:
for season, df in merge.items():
    path = f"../CSV_files/Season_{season}/squad_player_stats.csv"
    df.to_csv(path, index=False)